In [1]:
%pip install requests

  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached certifi-2024.2.2-py3-none-any.whl (163 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
  Using cached idna-3.6-py3-none-any.whl (61 kB)
  Using cached urllib3-2.2.1-py3-none-any.whl (121 kB)

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import json
import os
import requests
import time
import io


def write_json(data, output_dir):
    for summoner_id, match in data:
        with open(os.path.join(output_dir, f"{summoner_id}.json"), 'w', encoding='utf-8') as file:
            json.dump(match, file, indent=4, ensure_ascii=False)


def summoner_id_generator(summoner_id_path, output_dir):
    num_of_file = len(os.listdir(output_dir))

    with open(summoner_id_path) as json_file:
        json_data = json.load(json_file)
        for i in json_data[num_of_file:]:
            summoner_id = i['summoner_id']
            yield summoner_id


def crawling_matches(headers, summoner_id_generator):
    for summoner_id in summoner_id_generator:
        print(f"start: {summoner_id}")
        url = f"https://www.op.gg/api/v1.0/internal/bypass/games/kr/summoners/{summoner_id}?&limit=20&hl=ko_KR&game_type=soloranked"
        resp = requests.get(url, headers=headers)

        if resp.status_code == 200:
            user_json = json.loads(resp.text)
            yield summoner_id, user_json['data']
            
        elif resp.status_code == 404:
            user_json = json.loads('[{"resp.status_code": 404}]')
            yield summoner_id, user_json

        elif resp.status_code == 429:
            sleep_time = 1
            while resp.status_code == 429:
                time.sleep(sleep_time)
                sleep_time += 1

                print(f'sleep {sleep_time}')      
                resp = requests.get(url, headers=headers)
        
        else:
            print(f"unexpected response : {resp.status_code}")

        

In [18]:
if __name__ == "__main__":
    tier = 'emerald'
    summoner_id_path = "./summorer_ids"
    output_dir = "/home/piddle/hdd/matches"
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:122.0) Gecko/20100101 Firefox/122.0',}
    
    if not os.path.exists(os.path.join(output_dir, tier)):
        os.makedirs(os.path.join(output_dir, tier))
        
    summoner_id = summoner_id_generator(os.path.join(summoner_id_path, tier + '.json'), os.path.join(output_dir, tier))
    matches = crawling_matches(headers, summoner_id)
    write_json(matches, os.path.join(output_dir, tier))

start: uw1o5L6V-jf3bHB2UBKFYj5bhzCx9mi78HBg3YPRHAynQZIHJMvF7vzwsA
start: VAmU13N5PT0cUiuK1ZVLxYnu4PqyX8G6qundMzkAzcXdGnPhSohitToVrg
start: Ho0K9O_GEmyEPXwvA5U-NoO_EEa-pkKxprcFo6x6HUSZZXRBYyNurkcwPw
start: Bc1PJpMB9DrX2eWfg4PSowOCXtulq68V26ZrDMgov1r8dN82kNnzPusTEQ
start: G-5VNkKDjHQrA3LZjRpWHt_sFRXUCietx4mTWCOn9Fvf5FZik_OLgagJPA
start: CrTsLim2FnyYRSVt5Gj-FSvT2C_OMBl4OuMr9xS0nJNBaz7hhWEFqmazpg
start: OrPBWgFz7YGSGyo8Bqjkt9a1a0BFsPPguFrQu-8zwvp46YQHQGU0n_Z7Fw
start: nb53BLGskjwTKPoeDrK-ugVmCp1pseGDq9kX8i995631oqHU
start: t2maQmrPMbB5xtub5rnzoEuiStJ1_Qd_nEjlj7sbDHBzJDbb
start: F2QjcUCpkcdEmLTXWQXHGA1h107mdl56YyrRyrY6C0CFvvI
start: 6MyF6GZqR4qXdhuzbVHq-iS3q2PvA3qcHW0jvyP1p5F1juk
start: -UJZTfnQh7j-3241vBz-3-_-66GceL5mROwdlbCJ6GOPxFY
start: bshl_eg0djOEQQ8Xic-WD0VUd5LaM7u1jakQYuBZBPMQB4I
start: lQbNTZ8dp2wpkvJCaowgVEWKFg5hfdzV0lx6KeSl1PqZ3t8
start: DtGb4i-r_e6niObMV7mEZfHyE7AKZha8KSLBo6JdI8-tpMA
start: 1f3xumrMA3fyC7FrOe1GAb1iqXYdt9TWgLUU8ciLg6v9Ou0
start: xidm_Zxvo7dtcIMmtA3-7lQFVVz_xDbf6i